In [4]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
from functools import partial
import random
from typing import Dict, Tuple, Callable
from PIL import Image, UnidentifiedImageError
from typing import List, Optional
import pypdf
import orjson
import jsonlines     
import fitz  # PyMuPDF
from transformers import (
    LongformerModel,
    LongformerTokenizer,
    LongformerConfig,
    PreTrainedTokenizerFast,
    StoppingCriteria,
    StoppingCriteriaList,
    MBartConfig,
    MBartForCausalLM,
)


os.environ["CUDA_VISIBLE_DEVICES"]="2"
import pickle


In [12]:
pdf_path = "/mnt/NAS/patidarritesh/5_page_pdfs/0801.3623.pdf"
paper_id = pdf_path.split('/')[-1][:-4]
txt_path = f"/mnt/NAS/patidarritesh/Pdf_2_LaTeX_v2_LONGFORMER/pdf_2_tex/dataset/text_tensor/{paper_id}.pkl"
    
# if not os.path.exists(txt_path):
print("inside if, creating text tensor")
doc = fitz.open(pdf_path)
text = doc[0].get_text()
# print("text: ", text)

# Tokenize the text
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
text_input_ids = tokenizer(text, return_tensors="pt" , max_length=4096, truncation=True, padding=True)["input_ids"].squeeze(0)
print("Text Input IDS: ", text_input_ids)
print("Text Input IDS Shape: ", text_input_ids.shape)

# Serialize the dictionary to bytes
data = {"text_input_ids": text_input_ids}
serialized_data = pickle.dumps(data)

with open(txt_path, "wb") as f:
    f.write(serialized_data)



inside if, creating text tensor
Text Input IDS:  tensor([    0,   271,  1000,  1879,    35,  3669,  2663,     4,  3367,  1922,
          705,   134,  1437,   646,   700,   642,    12,  3463,   742,  1437,
          883,  1133,  2266, 50118, 28012,  6528,   783,     7, 22328,  1757,
            9, 46927,   862,  1640,   642, 47350,   642, 42484,   525,    43,
        43413, 16366,  1640,  1301, 42484, 48353, 47264,    43,    23, 47444,
        15113,    29,  5457,   134,     4,  5607,  2941,   846,     6,  1027,
           11, 50118, 43827,     4,  7161,     4,   211,  6121,     6, 13470,
        34972,    36, 32857,    43, 50118,  1640,   495,  1070,    35,   759,
          389,     6,   199,    43, 50118, 35765,   104,  1530,    35,   508,
            4,  3170,     4,   495,   571,     6,  1558,     4,  4531,     4,
         1864,   330,     6,  1570,     4,  3083,     4,   725,   642, 50118,
          133, 20104,     9, 46927,   862,  1640,   642, 47350,   642, 42484,
          525, 

In [13]:
# else:
print("inside else, opening text tensor")
if os.path.getsize(txt_path) > 0:  # Check if file is not empty
    with open(txt_path, "rb") as f:
        try:
            text_input_ids = pickle.load(f)
            text_input_ids = torch.tensor(text_input_ids['text_input_ids'])
        except Exception as e:

            # save to log text file instead of printing
            
            
            print(f"Error in {pdf_path}: {e}")
            text_input_ids = None  # or any other appropriate action
else:
    print(f"Error: {pdf_path} is empty.")
    text_input_ids = None 

# if(len(text_input_ids) > 4096):
#     text_input_ids = text_input_ids[:4096]



inside else, opening text tensor


/tmp/ipykernel_3339736/1244349852.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_input_ids = torch.tensor(text_input_ids['text_input_ids'])


In [14]:
text_input_ids.shape

torch.Size([723])